In [ ]:
# Install necessary packages
%pip install flask requests

In [8]:
# Import necessary libraries
from flask import Flask, request, jsonify
from datetime import datetime, timedelta
import requests


In [ ]:
# user database
users_db = {
    "mario.rossi": {"password": "password1", "role": "SMO", "email": "mario.rossi@gmail.com"},
    "luigi.bianchi": {"password": "password2", "role": "FFM", "email": "luigi.bianchi@gmail.com"},
    "antonio.tonarelli": {"password": "password3", "role": "FFM", "email": "antonio.tonarelli@gmail.com"},
    "RAG": {"password": "password4", "role": "System", "email": None},
    "KPIENGINE": {"password": "password5", "role": "System", "email": None},
    "PREPROCESSING": {"password": "password6", "role": "System", "email": None},
    
}

In [10]:
# Example of tokens database
tokens_db = {}

# Functions from authentication_db.py
def is_token_valid(token):
    if token in tokens_db:
        expiration = datetime.strptime(tokens_db[token], "%Y-%m-%dT%H:%M:%SZ")
        if expiration > datetime.now():
            return True
        else:
            del tokens_db[token]  # Remove expired token
    return False

def add_token_for_user(username):
    token = f"token{len(tokens_db)+1}"
    expiration = (datetime.now() + timedelta(minutes=5)).strftime("%Y-%m-%dT%H:%M:%SZ")
    tokens_db[token] = expiration
    return token

In [11]:
import threading
from werkzeug.serving import run_simple
# Initialize Flask app
app = Flask(__name__)


# Login endpoint to authenticate users and generate tokens
@app.route('/login', methods=["POST"])
def login():
    username = request.json.get("username")
    password = request.json.get("password")
    
    user = users_db.get(username)
    if user and user["password"] == password:
        token = add_token_for_user(username)
        return jsonify({"token": token}), 200
    return jsonify({"message": "Invalid credentials"}), 401

# Validate token endpoint
@app.route("/validate-token", methods=["POST"])
def validate_token():
    token = request.json.get("token")
    if not is_token_valid(token):
        return jsonify({"message": "Invalid or expired token"}), 401
    return jsonify({"message": "Token is valid"}), 200




def run_flask_app():
    run_simple("127.0.0.1", 5000, app)


flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()